In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import db_helper
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import importlib
import n2v as N2V
import utils
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import sys
import random
from gensim.models import Word2Vec

In [25]:
PROJECT_PATH='/home/hmenn/Workspace/CSE496_Node2Vec_2018'

engDateTag= {
    'month_tag':'MONTH',
    'year_tag':'YEAR',
    'day_tag':'DAY',
    'month_name':{1:"JAN",2:"FEB",3:"MAR",4:"APR",
          5:"MAY",6:"JUN",7:"JUL",8:"AUG",
         9:"SEP",10:"OCT",11:"NOV",12:"DEC"}
}

trDateTag = {
    'month_tag':'AY',
    'year_tag':'YIL',
    'day_tag':'GUN',
    'month_name':{1:"OCA",2:"SUB",3:"MAR",4:"NIS",
          5:"MAY",6:"HAZ",7:"TEM",8:"AGU",
         9:"EYL",10:"EKI",11:"KAS",12:"ARA"}
}

In [16]:
trDateTags['month_name'][2]

'SUB'

#### If you don't have json data, read data from db and then convert them into json to easy load-save. Reading data from db is not useful.

In [ ]:
"""
db = db_helper.DBHelper()
db.connectMysql(user="root",passwd="Hasan5695*",db="SocialMediaDB")
tweets = db.getTweetsFromMysql("TweetsShort")
print(tweets[0])
db_helper.saveTweetsAsJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json', tweets)
"""

In [18]:
# USE this if you have json data
tweets = db_helper.getTweetsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json')
#tweets.sort(key=lambda t: t.date)
allFriendships = db_helper.getAllFriendsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/friends.json')

In [19]:
print('TotalTweet#:',len(tweets))
print(tweets[0],"\n",tweets[len(tweets)-1])

print('\nExample AllFriendShips:\n',allFriendships[:5],allFriendships[100:105])

TotalTweet#: 93753
############ Tweet: ############ 
SendByScreenName:cakrr_ 
UserID:u1623755515 
TweetID:t763141405766250496 
InReplyToStatusID:t763141168771244032 
InReplyToUserID:u757176735158923264 
Date:2016-08-10 01:34:24 
Text:@Wesleyoloji10 OGLAK HAZİNE ARIYOR AQ:das:asd 
 
 ############ Tweet: ############ 
SendByScreenName:ahmet_iyimaya 
UserID:u3228821009 
TweetID:t851752159603568640 
InReplyToStatusID:t-1 
InReplyToUserID:u-1 
Date:2017-04-11 14:01:54 
Text:Yenimahalle'de İlçe teşkilatımız ile birliktetyiz. #Referandum çalışmalarımız tüm hızıyla devam ediyor. #Anayasa https://t.co/FOTZWbki2e 


Example AllFriendShips:
 [{'UserID': 'u100341775', 'FriendID': 'u1014141614'}, {'UserID': 'u100341775', 'FriendID': 'u120840897'}, {'UserID': 'u100341775', 'FriendID': 'u1494034783'}, {'UserID': 'u100341775', 'FriendID': 'u154236788'}, {'UserID': 'u100341775', 'FriendID': 'u155534196'}] [{'UserID': 'u129580191', 'FriendID': 'u63110096'}, {'UserID': 'u129580191', 'FriendID': 'u7161972

### Load NER Model

In [20]:
%%time
sys.path.append(PROJECT_PATH + '/src/tagger')

from loader import prepare_sentence
from utilsTagger import create_input, iobes_iob, iob_ranges, zero_digits
from model import Model

modelPath = PROJECT_PATH+'/dataset/NER_MODEL/tr2'
print('NER_ModelPath:',modelPath)

nerModel = Model(model_path=modelPath)
parameters = nerModel.parameters

# Load reverse mappings
word_to_id, char_to_id, tag_to_id = [
    {v: k for k, v in x.items()}
    for x in [nerModel.id_to_word, nerModel.id_to_char, nerModel.id_to_tag]
]

# Load the model
_, f_eval = nerModel.build(training=False, **parameters)
nerModel.reload()

print('Model Parameters:',parameters)

def extract_ner(line):
    if line:
        # Ahmet'in Turkiye'ye gidisi icin-> [Ahmet,in,Turkiye,ye,gidisi,icin]
        words_ini = line.rstrip().replace("'"," ").split()
        
        # Lowercase sentence
        if parameters['lower']:
            line = line.lower()
        # Replace all digits with zeros
        if parameters['zeros']:
            line = zero_digits(line)
        words = line.rstrip().replace("'"," ").split()
        # Prepare input
        sentence = prepare_sentence(words, word_to_id, char_to_id,
                                    lower=parameters['lower'])
        input = create_input(sentence, parameters, False)
        # Decoding
        if parameters['crf']:
            y_preds = np.array(f_eval(*input))[1:-1]
        else:
            y_preds = f_eval(*input).argmax(axis=1)
            
        y_preds = [nerModel.id_to_tag[y_pred] for y_pred in y_preds]
        # Output tags in the IOB2 format
        if parameters['tag_scheme'] == 'iobes':
            y_preds = iobes_iob(y_preds)
        
        #print('word_ini:', words_ini)
        #print('preds:',y_preds)
        return words_ini, y_preds
    return None

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


NER_ModelPath: /home/hmenn/Workspace/CSE496_Node2Vec_2018/dataset/NER_MODEL/tr2
Compiling...
Model Parameters: OrderedDict([('tag_scheme', 'iobes'), ('lower', False), ('zeros', False), ('char_dim', 25), ('char_lstm_dim', 25), ('char_bidirect', True), ('word_dim', 300), ('word_lstm_dim', 100), ('word_bidirect', True), ('pre_emb', 'dataset/wiki.tr.vec'), ('all_emb', False), ('cap_dim', 0), ('crf', True), ('dropout', 0.5), ('lr_method', 'adam')])
CPU times: user 7.02 s, sys: 1.2 s, total: 8.21 s
Wall time: 29.1 s


In [21]:
for i, t in enumerate(tweets[600:610]):
    words, ners = extract_ner(t.text)
    written = False
    for w,n in zip(words, ners):
        if n != 'O':
            if not written:
                print('Text:',t.text)
                print('NER:',end=' ')
                written = True
            w = w.lower()
            print(w,n,end=', ')
    if written:
        print('\n#######')

Text: @muratsagman klasik sağman yorumu tam 12 den bravo abi kısa bir değerleme yapsanız usdtry ve bist için teşekkür ederim
NER: @muratsagman B-MISC, 
#######
Text: #Rio2016 Erkekler Basketbol yarı finalinde Sırbistan, Avustralya'yı 87-61 mağlup etti ve finalde ABD'nin rakibi oldu https://t.co/icYSiVB4Ey
NER: basketbol B-MISC, avustralya B-LOC, 
#######
Text: #SporxNBA NBA tarihinde quadruple-double yapan kaç oyuncu var? Cevabı burada: https://t.co/fDZCGFpyH3
NER: oyuncu B-MISC, 
#######
Text: RT @abdullahciftcib: Fitch, Türkiye'nin kredi notu görünümünü durağandan negatife çekmiş.
Anlamı: J.Biden ile görüşme olumlu geçmez ise fin…
NER: türkiye B-LOC, 
#######


In [28]:
def create_all_graph(tweets, allFriendships, dateTag):
    G1 = nx.Graph()
    G1.add_node(dateTag['year_tag'])
    #G1.add_node(dateTag['month_tag'])
    #G1.add_node(dateTag['day_tag'])

    prevYear = None
    prevMonth = None
    prevDay = None

    prevYear = tweets[0].date.year
    prevMonth = tweets[0].date.month
    prevDay = tweets[0].date.day
        
    for i,tweet in enumerate(tweets):
        if (i+1) %1000 == 0:
            print(i,' th tweet was added.')
                
        ################# ADD DATE NODES #################
        G1.add_edge(dateTag['year_tag'],str(tweet.date.year),weight=1.0)
        if(prevYear < tweet.date.year):
            G1.add_edge(str(prevYear),str(tweet.date.year),weight=1.0)
            prevYear = tweet.date.year
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
          
        monthFormat = "{month}.{year}".format(month=tweet.date.month,year=tweet.date.year)
        G1.add_edge(str(tweet.date.year),monthFormat,weight=1.0)
        
        if(prevMonth < tweet.date.month):
            prevMonthFormat = "{month}.{year}".format(month=prevMonth,year=tweet.date.year)
            G1.add_edge(prevMonthFormat,monthFormat,weight=1.0)
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
        
        #G1.add_edge(MONTH_TAG,monthFormat,weight=1.0)
        fullDateFormat = "{day}.{month}.{year}".format(day=tweet.date.day,month=tweet.date.month,year=tweet.date.year)
        if prevDay < tweet.date.day:
            prevFullDateFormat = "{day}.{month}.{year}".format(day=prevDay,month=tweet.date.month,year=tweet.date.year)
            G1.add_edge(prevFullDateFormat,fullDateFormat,weight=1.0)
            prevDay = tweet.date.day
        G1.add_edge(monthFormat,fullDateFormat,weight=1.0)
        
        ################# ADD TWEET INFORMATIONS LIKE TEXT, USER #################
        # date -> tweet
        G1.add_edge(fullDateFormat,tweet.tweetID,weight=1.0)
        
                
        # if user will be added first time, add friend connections
        if not G1.has_node(tweet.userID):
            friends = [x['FriendID'] for x in allFriendships if x['UserID']==tweet.userID]
            # now check if friends tweeted before
            userFriends = []
            for friend  in friends:
                for t in tweets:
                    if t.userID==friend:
                        userFriends.append((tweet.userID,friend, {'weight':1.0}))
                
            # add friendship connections
            G1.add_edges_from(userFriends)
            
        # user -> tweet connections were established
        G1.add_edge(tweet.userID, tweet.tweetID,weight=1.0)
                 
        # if tweet replied and db has actual(replied tweet, add to graph)
        # cevap verilen tweet db de ise, o tweete baglanti ekle
        for i in tweets:
            if i.tweetID == tweet.inReplyToStatusID:
                G1.add_edge(tweet.tweetID, tweet.inReplyToStatusID,weight=1.0)
                #print('Replied:',tweet.tweetID, '<->',tweet.inReplyToStatusID)
                
        ################# ADD NER's #################
        
        words, ners = extract_ner(tweet.text)
        written=False
        for w,n in zip(words, ners):
            if n != 'O':
                if not written:
                    print('Text:',tweet.text)
                    print('NER:',end=' ')
                    written = True
                w = w.lower()
                print(w,n,end=', ')

                G1.add_edge(tweet.tweetID, w,weight=1.0)
        if written:
            print('\n#######')
        
    return G1

In [23]:
dataset = tweets[0:100]
dataset += tweets[3000:3050]
dataset += tweets[5000:5100]
dataset += tweets[8000:8050]
len(dataset)

300

In [29]:
%%time
graph = create_all_graph(dataset, allFriendships, dateTag=trDateTag)
print("Number of edges: ",graph.number_of_edges())
print("Number of nodes: ",graph.number_of_nodes())

Text: Sn.CB'nımızın çağrısıyla 3 banka konut kredisi faiz oranlarını ℅1'in altına çekti..Bu olumlu bir adım.

Ancak şişen piyasa nasıl düşecek?
NER: faiz B-MISC, 
#######
Text: Konut kira/satışları düşünüldüğünde piyasa öylesine şişti ki.Örneğin İstanbul'da 1000 liranın altında ev kiralamanız nerdeyse mümkün değil!
NER: i̇stanbul B-LOC, 
#######
Text: BDDK/Akben: Piyasa şartlarıyla birlikte kredi faiz oranlarının daha makul seviyelere geleceğini düşünüyoruz
NER: faiz B-MISC, 
#######
Text: #SporxNBA NBA'in en durdurulmaz ve hırslı süperstarı Westbrook'tan 10 en iyi hareket! https://t.co/E2svcVWyzv https://t.co/FHaOKk5qy1
NER: westbrook B-PER, 
#######
Text: #Rio2016 Erkekler Basketbol B grubunda İspanya, Litvanya'yı 109-59 mağlup etti. https://t.co/UYTKSwQfEn
NER: basketbol B-MISC, litvanya B-LOC, 
#######
Text: İsil dun aksam dunya basketbol tarihine gecmis..
NER: basketbol B-MISC, 
#######
Text: #Rio2016 Erkekler Basketbol A grubunda ABD, Fransa'yı 100-97 mağlup ederek grubu namağlup

Text: RT @YUNInvest: Resmi açıklama yok ancak @JulideYigitturk'ün haberine göre Borsa İstanbul'da işlem saatleri değişiyor. 

https://t.co/KsAHM3…
NER: i̇stanbul B-LOC, 
#######
Text: RT @turgaydemirr: Penaltı uydurmak yrtmez Fırat iki Karabüklü'yü de atarsa AY'nin gözüne girer.
NER: fırat B-MISC, 
#######
Text: RT @pilavistt: Güntekin Onay: Galatasaraylı futbolcular roma olimpiyat stadında polis dövüyor yapmayın çocuklar...

Unutursak kalbimiz kuru…
NER: futbolcular B-MISC, 
#######
Text: RT @pilavistt: Güntekin Onay: Galatasaraylı futbolcular roma olimpiyat stadında polis dövüyor yapmayın çocuklar...

Unutursak kalbimiz kuru…
NER: futbolcular B-MISC, 
#######
Text: Ey gidi Olimpiyat stadinda Ahmet Hassan'in 2 metre uzerinden gecen topa calinan penalti..
NER: ahmet B-PER, hassan I-PER, 
#######
Text: Bebek çantasından çıkanlar şaşkına çevirdi! https://t.co/3U5apEqTEH https://t.co/ZBDxgU2vp3
NER: bebek B-MISC, 
#######
Text: RT @SMEYDAN: Akp-cemaat ittifakının temel nedenlerinden biri 

Text: RT @ntvspot: SON DAKİKA | Fenerbahçe, Hatem Ben Arfa için oyuncunun kulübü ve kendisiyle görüşmelere başlandığını anasının amına bildirdi.…
NER: oyuncunun B-MISC, 
#######
Text: Eduardo vargas geliyordu hani ona da burun kıvırmıştım ama razıyım şimdi
NER: eduardo B-PER, 
#######
Text: RT @1905GSFANS: Ahmet Çalık son üç sezonda Süper Lig'de en fazla maça çıkan 2. (75), en fazla uzaklaştırma yapan 1. savunma oyuncusu (472)…
NER: oyuncusu B-MISC, 
#######
Text: RT @turgaydemirr: Mitroviç'i sosyal medyada  savunan, gazeteci maskesi takmış leş kargalarını görünce kafam çok karışıyor çoook.
NER: gazeteci B-MISC, 
#######
Number of edges:  865
Number of nodes:  484
CPU times: user 59.3 s, sys: 1min 46s, total: 2min 46s
Wall time: 1min 4s


In [ ]:
%%time
plt.figure(1, figsize=(30,30))
nx.draw_kamada_kawai(graph,node_size=5000,font_size=50,with_labels=True, width=3, edge_color="b", arrows=True, arrowsize=30)
plt.savefig(PROJECT_PATH+"/outputs/twitter_nx.pdf")
plt.show()

In [30]:
# save and read edge list
#nx.write_edgelist(graph, path=PROJECT_PATH+"/outputs/grid.edgelist", delimiter=":")
#graph = nx.read_edgelist(path="grid.edgelist", delimiter=":")

**If you have existing trained model, use load functionality.**

In [31]:
%%time
graphN2V = N2V.Graph(graph, is_directed = False, p=1.0, q=1.0)
graphN2V.preprocess_transition_probs()
walks = graphN2V.simulate_walks(1000, 80)
print('Learning Embeddings...')
model = N2V.learn_embeddings(walks,"twitter")

Walk iteration:
1000 / 1000
Learning Embeddings...
CPU times: user 38min 46s, sys: 3.12 s, total: 38min 49s
Wall time: 15min 46s


In [32]:
# Load model from existing file
# model = Word2Vec.load(PROJECT_PATH+'/outputs/twitter.model')

In [33]:
graph['YIL'] #,graph[]

AtlasView({'2016': {'weight': 1.0}, '2017': {'weight': 1.0}})

In [ ]:
utils.draw_tsne(model, path=PROJECT_PATH + '/outputs/twitter_tsne.pdf', fig_size=(50,50))

### Cosine Similarities for Times

In [37]:
graph['u1623755515']

AtlasView({'u1278512364': {'weight': 1.0}, 'u1499266621': {'weight': 1.0}, 'u1829708546': {'weight': 1.0}, 'u18426124': {'weight': 1.0}, 'u23186079': {'weight': 1.0}, 'u2424673684': {'weight': 1.0}, 'u4327240396': {'weight': 1.0}, 'u4856670141': {'weight': 1.0}, 'u583863506': {'weight': 1.0}, 'u61171559': {'weight': 1.0}, 'u732590069681393664': {'weight': 1.0}, 'u768542424': {'weight': 1.0}, 't763141405766250496': {'weight': 1.0}, 'u565848357': {'weight': 1.0}, 'u471859539': {'weight': 1.0}, 't819588562698010624': {'weight': 1.0}})

In [40]:
column_names = ['2016', '8.2016', '10.8.2016', "16.8.2016", '10.2016','29.10.2016','30.10.2016','2017','1.2017', '12.1.2017']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,2016,8.2016,10.8.2016,16.8.2016,10.2016,29.10.2016,30.10.2016,2017,1.2017,12.1.2017
2016,1.000000,0.653825,0.509933,0.174226,0.754052,0.357579,0.227544,0.764847,0.571097,0.179766
8.2016,0.653825,1.000000,0.749489,0.338116,0.591549,0.300241,0.218591,0.435226,0.331710,0.184006
10.8.2016,0.509933,0.749489,1.000000,0.265180,0.472819,0.215346,0.216337,0.358742,0.355489,0.255866
16.8.2016,0.174226,0.338116,0.265180,1.000000,0.156207,0.124332,0.194046,0.088237,0.133471,0.172880
10.2016,0.754052,0.591549,0.472819,0.156207,1.000000,0.579656,0.454672,0.485708,0.348812,0.158056
29.10.2016,0.357579,0.300241,0.215346,0.124332,0.579656,1.000000,0.419442,0.248395,0.215501,0.217186
30.10.2016,0.227544,0.218591,0.216337,0.194046,0.454672,0.419442,1.000000,0.188774,0.211827,0.238516
2017,0.764847,0.435226,0.358742,0.088237,0.485708,0.248395,0.188774,1.000000,0.823800,0.402279
1.2017,0.571097,0.331710,0.355489,0.133471,0.348812,0.215501,0.211827,0.823800,1.000000,0.647309
12.1.2017,0.179766,0.184006,0.255866,0.172880,0.158056,0.217186,0.238516,0.402279,0.647309,1.000000


### Cosine Similarities for Users

In [41]:
column_names = ['u1623755515','u565848357','u700397072063795200','u296406287']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,u1623755515,u565848357,u700397072063795200,u296406287
u1623755515,1.000000,0.201955,0.242478,0.216975
u565848357,0.201955,1.000000,0.155391,0.405474
u700397072063795200,0.242478,0.155391,1.000000,0.090024
u296406287,0.216975,0.405474,0.090024,1.000000


### Cosine Similarities for Tweets

t765466344707989504, t765582802343657472 ve t765648654883512321 aynı gun atılmıstır ama ılk ıkısı extra olarak basketboldan konusmuslardır.

In [42]:
column_names = ['t765562268767678464','t763141405766250496','basketbol','t765466344707989504','t765582802343657472','t765648654883512321']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,t765562268767678464,t763141405766250496,basketbol,t765466344707989504,t765582802343657472,t765648654883512321
t765562268767678464,1.000000,0.154652,0.222707,0.350529,0.357823,0.452084
t763141405766250496,0.154652,1.000000,0.267026,0.142732,0.295929,0.140562
basketbol,0.222707,0.267026,1.000000,0.651889,0.668917,0.353605
t765466344707989504,0.350529,0.142732,0.651889,1.000000,0.636400,0.498252
t765582802343657472,0.357823,0.295929,0.668917,0.636400,1.000000,0.425720
t765648654883512321,0.452084,0.140562,0.353605,0.498252,0.425720,1.000000


In [43]:
%%time
words, preds = extract_ner("Hasan Türkiye'ye taşındı.")
print(words, preds)

['Hasan', 'Türkiye', 'ye', 'taşındı.'] ['B-PER', 'B-LOC', 'O', 'O']
CPU times: user 117 ms, sys: 95.8 ms, total: 213 ms
Wall time: 198 ms


In [44]:
import gc
gc.collect()

50003